In [1]:
import pandas
import json
import h2o
from h2o.automl import H2OAutoML
import os
from pathlib import Path

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /home/ramses/Documentos/proyects/web-mining/NewsNabber/.venv/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpnh8xuurp
  JVM stdout: /tmp/tmpnh8xuurp/h2o_ramses_started_from_python.out
  JVM stderr: /tmp/tmpnh8xuurp/h2o_ramses_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Argentina/Buenos_Aires
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_ramses_7z77lm
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.914 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [4]:
def load_articles() -> list:
    articles = []
    for file_path in os.listdir("./data"):
        try:
            data_page = open(Path("data", file_path))
            articles_by_page = json.load(data_page)
            articles = articles + articles_by_page
        except:
            print(f"No se pudo cargar el arcivo: {file_path}")
    
    return articles

In [5]:
def preprocess_text(text: str):
    return []

In [7]:
dataframe = pandas.DataFrame(load_articles())

,date,description,title,category
0,2023-08-25T19:46:52-03:00,A la espera de los\nanuncios del ministro de E...,,economia
1,2023-08-25T11:26:52-03:00,Un supermercado ubicado en la ciudad de La Pla...,,economia
2,2023-08-25T19:05:48-03:00,"El dólar blue cerró su cotización a $720,00 pa...",,economia
3,2023-08-25T00:01:00-03:00,Las ventas en supermercados medidas a precios ...,,economia
4,2023-08-25T12:50:13-03:00,El ministro de Economía y candidato a presiden...,,economia
...,...,...,...,...
171,2023-08-05T00:01:00-03:00,La inflación comenzó a acelerarse nuevamente e...,,economia
172,2023-08-05T00:01:00-03:00,El Banco Central cerró la jornada de este vier...,,economia
173,2023-08-05T00:01:00-03:00,La AFIP impuso un ajuste de más 3.000 millones...,,economia
174,2023-08-05T08:55:50-03:00,En una maniobra que se había guardado celosame...,,economia


In [10]:
dataframe.groupby(['date', 'category']).count()

,,description,title
date,category,,
2023-08-02T00:01:00-03:00,economia,4,4
2023-08-02T16:14:38-03:00,economia,1,1
2023-08-03T00:01:00-03:00,economia,5,5
2023-08-05T00:01:00-03:00,economia,6,6
2023-08-05T08:55:50-03:00,economia,1,1
...,...,...,...
2023-08-25T17:04:45-03:00,el-pais,1,1
2023-08-25T19:05:48-03:00,economia,1,1
2023-08-25T19:42:50-03:00,el-pais,1,1


In [ ]:
def generate_model(df_train: pandas.DataFrame, df_test: pandas.DataFrame):
    train = h2o.H2OFrame(df_train)
    test = h2o.H2OFrame(df_test)

    x = train.columns
    y = "category"
    x.remove(y)

    train[y] = train[y].asfactor()
    test[y] = test[y].asfactor()

    auto_machine_learning = H2OAutoML(max_models=20, seed=1, max_runtime_secs=60*2)
    auto_machine_learning.train(x=x, y=y, training_frame=train)